# Laboratorio de Introducción al Procesamiento de Lenguaje Natural 2017

#### Número de grupo: 13
#### Integrantes:
- Giovani Rondán, CI: 4.528.997-6
- Santiago Behak, CI: 5.019.450-0

## 1- Importación de los tweets.

Comenzaremos importando los tweets provenientes del archivo "corpus_humor_training.csv" usando la librería Pandas:

In [1]:
import pandas
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
import re
import freeling
from pylab import *
import numpy as np
from nltk.corpus import stopwords
import IPython.display as disp

corpus = pandas.read_csv("corpus_humor_training.csv",encoding='utf-8')

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


IOError: File corpus_humor_training.csv does not exist

Los registros de este corpus están compuestos por varios datos además del propio texto del tweet. A continuación mostraremos la estructura del corpus y algunos de los tweets (que se encuentran en el atributo "text") a modo de ejemplo:

In [ ]:
print(corpus.columns)
print "\n"
for text in corpus['text'][:7]:
    print(text + '\n')

A continuación porocedemos a analizar las características del corpus obtenido, para esto obtendremos algunos datos básicos tales como la cantidad total de tweets, la cantidad de atributos de los que disponemos, la cantidad de calificaciones de los 10 tweets más calificados y la cantidad total de calificaciones.

In [ ]:
cantTweets = len(corpus)
cantAtributos = len(corpus.columns)

# Imprimimos la cantidad total de tweets y la cantidad de atributos
print ("Cantidad de Tweets en el corpus: " + str(cantTweets))
print ("Cantidad de atributos en el corpus: " + str(cantAtributos))

# Imprimimos la cantidad de calificaciones de los 10 tweets mas calificados y la cantidad total de calificaciones
totalCalificaciones = 0
contador = 0
corpus["cantCalificaciones"] = [0]*len(corpus)
for i in range(0, 12106):
    calificacionesTweet =corpus.loc[i, "n"] + corpus.loc[i, "1"]  + corpus.loc[i, "2"] + corpus.loc[i, "3"] + corpus.loc[i, "4"] + corpus.loc[i, "5"]
    totalCalificaciones += calificacionesTweet
    corpus.loc[i, "cantCalificaciones"] = calificacionesTweet

print ("Lista de los diez tweets con más calificaciones:\n")
disp.display(corpus.sort_values(by = ['cantCalificaciones'], ascending = False).head(10))

Otro dato que puede resultar interesante es la cantidad de calificaciones por valor (las calificaciones no humorísiticas serán contadas con el 0).

In [ ]:
%matplotlib inline

# Se realiza una gráfica de cantidad de comentarios en función de su clasificación
calificacionesPorValor = [corpus["n"].sum(), corpus["1"].sum(), corpus["2"].sum(), corpus["3"].sum(), corpus["4"].sum(), corpus["5"].sum()]
valoresCalificaciones = ["0", "1", "2", "3", "4", "5"]
pos = arange(len(valoresCalificaciones)) + 0.5 

figure(1)
barh(pos,calificacionesPorValor, align='center')
yticks(pos, valoresCalificaciones)
xlabel('Cantidad de calificaciones')
ylabel(u'Calificación')
title(u'Cantidad de calificaciones por valor')
grid(True)
show()

Como se puede ver en la gráfica la mayoría de las calificaciones corresponden al valor 0, o sea, como calificaciones no humorísticas.

## 2- Preprocesmiento de los datos

En primer lugar procederemos a eleminar las columnas del corpus que consideramos innecesarias. Eliminaremos las columnas tanto de la id del tweet como la de la id del usuario. La id del tweet es un número autogenerado que no aporta información relevante ya que es completamente aleatoria y la id del usuario, si bien puede llegar a relacionarse con usuarios que tienden a emitir tweets de carácter más o menos humorístico, no es un factor concluyente para determinar el nivel de humor de un tweet particular (un mismo usuario puede hacer tweets humorísticos y no humorísticos).

In [ ]:
if "id" in corpus.columns:
    del corpus["id"]
if "account_id" in corpus.columns:
    del corpus["account_id"]
print corpus.columns


Una vez filtradas las columnas del corpus nos disponemos a filtrar los tweets que tienen menos de tres calificaciones dado que los mismos no cuentan con una cantidad significativa de calificaciones como para ser evaluados. Además se eliminarán los hashtags de los textos de los tweets como se pide en la letra y agregaremos una nueva columna que determina si un tweet es humorístico en función del número de calificaciones no humorísticas en relación al total de calificaciones del tweet. Si la cantidad de calificaciones humorísticas es mayor o igual a la suma del resto de calificaciones el tweet se considerará no humorístico.

In [ ]:
corpus["humoristico"] = [False]*len(corpus)
corpus_filtrado = pandas.DataFrame(columns = ['text', 'n', '1', '2', '3', '4', '5', 'humoristico', 'cantCalificaciones'])
#corpus_filtrado = corpus[corpus.n + corpus.columns[4] + corpus.columns[5] + corpus.columns[6] + corpus.columns[7] + corpus.columns[8] >= 3]
total = 0
for i in range(0, 12106):
    contador = corpus.loc[i, "cantCalificaciones"]
    #eliminamos los hashtags
    corpus.loc[i, "text"] = re.sub(r"#\S+\s*", "", corpus.loc[i, "text"])
    #definimos si un tweet es humoristico o no segun los votos
    if(contador/2 >= corpus.loc[i, "n"]):
        corpus.loc[i, "humoristico"] = True
    #filtramos los tweets que tienen menos de 3 votos
    if contador >= 3:
        corpus_filtrado.loc[total] = [corpus.loc[i, "text"], corpus.loc[i, "n"], corpus.loc[i, "1"], corpus.loc[i, "2"], corpus.loc[i, "3"], corpus.loc[i, "4"], corpus.loc[i, "5"], corpus.loc[i, "humoristico"], corpus.loc[i, "cantCalificaciones"]]
        total += 1
        
#columna 3 -> n, 4 -> 1, 5 -> 2, 6 -> 3, 7 -> 4, 8 -> 5

disp.display(corpus_filtrado.loc[0:10, :])
print "Cantidad de tweets que quedan en el corpus luego del filtrado: " + str(len(corpus_filtrado))

## 3- Separación de los datos en conjunto de train y test

A continuación dividiremos el corpus restante en un conjunto de train y en otro de test. En principio usaremos un 80% de los datos para el entrenamiento y un 20% para el testeo.

In [ ]:
corpus_train, corpus_test = train_test_split(corpus_filtrado, test_size=0.2)

print ("Cantidad de tweets en el conjunto de entrenamiento: " + str(len(corpus_train)))
print ("Cantidad de tweets en el conjunto de testeo: " + str(len(corpus_test)))

## 4- Clasificador binario con tokenizador y POS tag de Freeling

Una vez separados el conjunto de train y test procedemos a tokenizar los tweets provenientes del conjunto train. Para esto usaremos la librería Freeling y NLTK

### 4.1- Tokenización

In [ ]:
from subprocess import Popen, PIPE, STDOUT

os.environ["ANALYZER"] = "/home/santiago/Descargas/FreeLing-4.0/src/main"
os.environ["FREELINGSHARE"] = "/home/santiago/Descargas/FreeLing-4.0/data/es"
comando = os.environ["ANALYZER"] + "/analyzer -f " + "/home/santiago/Descargas/FreeLing-4.0/data"

def POS_tagging(datos, largo):
    listaTuplas = []
    
    # Retorna una lista de tuplas. 
    # Cada tupla posee un diccionario (dict) palabra-frecuencia del comentario y la clasificación asociada
    # En otras palabras [(dict1, clasificacion1),(dict2, clasificacion2), ... ]

    # Se recorren los comentarios y para cada uno de ellos se tokeniza con nltk
    for i in range(1, len(datos)):
        try:
            # Se crea el diccionario asociado al comentario
            dic = {}
            
            # Por cada palabra retornada de la tokenizacion del comentario
            p = Popen(comando, shell = True, stdout=PIPE, stdin=PIPE, stderr=STDOUT)
            stdout = p.communicate(input=datos.loc[i, "text"].encode('ascii','ignore'))[0]
            for linea in stdout.decode().split('\r\n'):
                print linea
                token = linea.split(' ')
                tag = token[2]
                palabra = token[0]
                print i
                print palabra
                
                if((tag[0:1] != 'F') and (tag[0:2] != 'RG') and (tag[0:2] != 'DP') and (tag[0:2] != 'DT') and (tag[0:2] != 'DE') 
                    and (tag[0:2] != 'DA') and (tag[0:1] != 'N') and (tag[0:2] != 'RG') and (tag[0:2] != 'PP') and (tag[0:2] != 'PD')
                    and (tag[0:2] != 'PX') and (tag[0:2] != 'PT') and (tag[0:2] != 'PR') and (tag[0:2] != 'PE') and (tag[0:1] != 'I')
                    and (tag[0:1] != 'S') and (tag[0:1] != 'Z') and (tag[0:1] != 'W')):

                    # Si la palabra está en el diccionario del comentario, se aumenta la frecuencia
                    # En caso contrario se la pone en el diccionario con valor 1
                    if(palabra.lower() in dic): 
                        dic[palabra.lower()] = dic[palabra.lower()] + 1
                    else:
                        dic[palabra.lower()] = 1

                    # Luego de tokenizado el comentario, se agrega una tupla a la lista que contendrá
                    # el diccionario de frecuencias y la clasificaion asociada al comentario
            listaTuplas.insert(i,(dic,datos.loc[i, "humoristico"]))
        except:
            print "explote"

    return listaTuplas

datos_test_tokenizados_pos_tagging = POS_tagging(corpus_train, len(corpus_filtrado))

In [ ]:
import os
for i in os.environ:
    print i

In [ ]:
for x in datos_test_tokenizados_pos_tagging[0][0]:
    print (str(x) + ':' + str(datos_test_tokenizados_pos_tagging[0][0][x]))